In [ ]:
import  dlc_bci as bci
import torch
from torch import optim
from torch import Tensor
from torch.autograd import Variable
from torch import nn
import numpy as np
from torch.nn import functional as F
from sklearn.linear_model import LogisticRegression

In [ ]:
train_input, train_target = bci.load(root =  './data')
print(str(type(train_input)), train_input.size())
print(str(type(train_target)), train_target.size())
test_input , test_target = bci.load(root =  './data', train = False)
print(str(type(test_input)), test_input.size())
print(str(type(test_target)), test_target.size())

In [ ]:
train_input = train_input.numpy().reshape(316,-1)
train_target = train_target.numpy()
test_input = test_input.numpy().reshape(100,-1)
test_target = test_target.numpy()

train_input.shape

In [ ]:
train_input[:,0]

In [ ]:
lr = LogisticRegression()
lr.fit(train_input, train_target)

lr.score(test_input, test_target)

In [ ]:
mean, std = train_input.mean(), train_input.std()
train_input.sub_(mean).div_(std)
test_input.sub_(mean).div_(std)

train_input = train_input.view(train_input.size(0),1,28,-1)
test_input = test_input.view(test_input.size(0),1,28,-1)

#train_input = train_input.view(train_input.size(0),-1)
#test_input = test_input.view(test_input.size(0),-1)

train_input, train_target = Variable(train_input), Variable(train_target)
test_input, test_target = Variable(test_input), Variable(test_target)


print(train_input.size())


In [ ]:
def create_deep_model():
    return nn.Sequential(
        nn.Linear(1400, 2000),
        nn.ReLU(),
        nn.Linear(2000, 2500),
        nn.ReLU(),
        nn.Linear(2500, 1)
        
    )

In [ ]:
class Net(nn.Module):
    def __init__(self, nb_hidden):
        super(Net, self).__init__()
        self.conv1 = nn.Conv2d(1, 32, kernel_size=5)
        self.conv2 = nn.Conv2d(32, 64, kernel_size=5)
        self.fc1 = nn.Linear(640, nb_hidden)
        self.fc2 = nn.Linear(nb_hidden, 1)

    def forward(self, x):
        x = F.tanh(F.max_pool2d(self.conv1(x), kernel_size=3, stride=3))
        x = F.tanh(F.max_pool2d(self.conv2(x), kernel_size=2, stride=2))
        #print(x.size())
        
        x = F.tanh(self.fc1(x.view(-1, 640)))
        x = self.fc2(x)
        return x

In [ ]:
class Net2(nn.Module):
    def __init__(self, nb_hidden):
        super(Net2, self).__init__()
        self.conv1 = nn.Conv2d(1, 32, kernel_size=5)
        self.conv2 = nn.Conv2d(32, 64, kernel_size=5)
        self.fc1 = nn.Linear(640, nb_hidden)
        self.fc2 = nn.Linear(nb_hidden, 2)

    def forward(self, x):
        x = F.tanh(F.max_pool2d(self.conv1(x), kernel_size=3, stride=3))
        x = F.tanh(F.max_pool2d(self.conv2(x), kernel_size=2, stride=2))
        #print(x.size())
        
        x = F.tanh(self.fc1(x.view(-1, 640)))
        x = self.fc2(x)
        return x

In [ ]:
def train_model(model, train_input, train_target, mini_batch_size, nb_epochs, eta):
    criterion = nn.CrossEntropyLoss()
    optimizer = optim.SGD(model.parameters(), lr = eta)

    for e in range(0, nb_epochs):
        sum_loss = 0
        for b in range(0, train_input.size(0), mini_batch_size):
            output = model(train_input.narrow(0, b, mini_batch_size))
            loss = criterion(output, train_target.narrow(0, b, mini_batch_size))
            model.zero_grad()
            loss.backward()
            optimizer.step()
            sum_loss += loss.data[0]
        print("epoch {} : {}".format(e,sum_loss))

def compute_nb_errors(model, data_input, data_target, mini_batch_size):

    nb_data_errors = 0

    for b in range(0, data_input.size(0), mini_batch_size):
        output = model(data_input.narrow(0, b, mini_batch_size))
        _, predicted_classes = torch.max(output.data, 1)
        for k in range(0, mini_batch_size):
            if data_target.data[b + k] != predicted_classes[k]:
                nb_data_errors = nb_data_errors + 1

    return nb_data_errors


In [ ]:
hidden = [100,150,200,250,300,350,400,450]
epochs = [50,100,150,200,250,300]

for eta in [1e-1, 1e-2, 1e-3]:
    for nb_hidden in hidden:
        model = Net2(nb_hidden)
        for nb_epochs in epochs:
            train_model(model, train_input, train_target, nb_epochs, eta)
            train_acc = 100 - compute_nb_errors(model, train_input, train_target, 4) / train_input.size(0) * 100
            test_accu = 100 - compute_nb_errors(model, test_input, test_target, 4) / test_input.size(0) * 100
            print("step {}, hidden {}, epoch {}, acc train {}%, acc test {}%".format(eta,nb_hidden, nb_epochs, train_acc, test_accu))

In [ ]:
#model = Net(250)
model = Net2(250)
#model = create_deep_model()
train_model(model, train_input, train_target, 4, 200, 1e-2)

In [ ]:
100 - compute_nb_errors(model, train_input, train_target, 4) / train_input.size(0) * 100

In [ ]:
100 - compute_nb_errors(model, test_input, test_target, 4) / test_input.size(0) * 100